In [1]:
import os
os.chdir('/nyb/yb_RiceSeg/RiceSeg/mmsegmentation') ## mmsegmentation 경로

In [2]:
output_dir = '/nyb/yb_RiceSeg/Output/RGB/model_output/knet'
config_path = '/nyb/yb_RiceSeg/RiceSeg/mmsegmentation/configs/'

In [3]:
import mmseg
print(mmseg.__version__)
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
import mmcv

from mmcv import Config
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
from mmseg.utils import get_device


# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.28.0
1.12.0 True


In [4]:
# nvcc version 
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.12.0 True
0.28.0


In [6]:
import os
cfg = Config.fromfile(config_path + 'knet/knet_s3_pspnet_r50-d8_8x2_512x512_adamw_80k_ade20k_rice.py')
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (512, 512)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(512, 512), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(512, 512),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

In [8]:
cfg.load_from = '/nyb/yb_RiceSeg/RiceSeg/pretrained_chkp/knet_s3_pspnet_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_054634-d2c72240.pth'
cfg.work_dir = '/nyb/yb_RiceSeg/Output/RGB/model_output/knet'
cfg.gpu_ids = range(1)
cfg.device = 'cuda'
cfg.seed = 0

In [10]:
from mmseg.datasets import build_dataset
datasets = [build_dataset(cfg.data.train)]

print(datasets[0].CLASSES)
print(datasets[0].PALETTE)

2022-11-08 16:52:34,431 - mmseg - INFO - Loaded 3246 images


('background', 'normal', 'doyeol', 'dobok', 'gyulju', 'bujin')
[[0, 0, 0], [8, 39, 245], [0, 229, 245], [225, 0, 26], [250, 0, 204], [245, 123, 0]]


In [12]:

from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Build the dataset
datasets = [build_dataset(cfg.data.train)]        #'/JH/jihye/mmsegmentation/configs/_base_/datasets/Rice_Sample.py'
classes = ('background','nomal', 'doyeol', 'dobok', 'gyulju', 'bujin')
palette = [[0,0,0], [8, 39, 245], [0, 229, 245], [225, 0, 26], [250, 0, 204], [245, 123, 0]]

# Build the detector
model = build_segmentor(cfg.model)               #, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

model.CLASSES = datasets[0].CLASSES
model.Palette = datasets[0].PALETTE

# Add an attribute for visualization convenience
#model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict(CLASSES=classes, PALETTE=palette))

2022-11-08 16:52:34,852 - mmseg - INFO - Loaded 3246 images
/nyb/yb_RiceSeg/RiceSeg/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
2022-11-08 16:52:52,585 - mmseg - INFO - Loaded 406 images
2022-11-08 16:52:52,588 - mmseg - INFO - load checkpoint from local path: /nyb/yb_RiceSeg/RiceSeg/pretrained_chkp/knet_s3_pspnet_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_054634-d2c72240.pth
2022-11-08 16:52:54,570 - mmseg - WARNING - The model and loaded state dict do not match exactly

size mismatch for decode_head.kernel_generate_head.conv_seg.weight: copying a param with shape torch.Size([150, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([6, 512, 1, 1]).
size mismatch for de

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 406/406, 13.9 task/s, elapsed: 29s, ETA:     0s

2022-11-08 16:56:03,951 - mmseg - INFO - per class results:
2022-11-08 16:56:03,977 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 79.88 | 86.13 |
|   normal   | 90.31 | 97.68 |
|   doyeol   |  nan  |  nan  |
|   dobok    |  0.0  |  0.0  |
|   gyulju   |  5.5  |  5.88 |
|   bujin    |  0.0  |  0.0  |
+------------+-------+-------+
2022-11-08 16:56:03,978 - mmseg - INFO - Summary:
2022-11-08 16:56:03,981 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 92.04 | 35.14 | 37.94 |
+-------+-------+-------+
2022-11-08 16:56:03,990 - mmseg - INFO - Iter(val) [406]	aAcc: 0.9204, mIoU: 0.3514, mAcc: 0.3794, IoU.background: 0.7988, IoU.normal: 0.9031, IoU.doyeol: nan, IoU.dobok: 0.0000, IoU.gyulju: 0.0550, IoU.bujin: 0.0000, Acc.background: 0.8613, Acc.normal: 0.9768, Acc.doyeol: nan, Acc.dobok: 0.0000, Acc.gyulju: 0.0588, Acc.bujin: 0.0000
2022-11-08 16:56:12,542 - mmse

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 406/406, 14.5 task/s, elapsed: 28s, ETA:     0s

2022-11-08 16:58:57,308 - mmseg - INFO - per class results:
2022-11-08 16:58:57,314 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 75.44 | 78.55 |
|   normal   | 89.26 | 98.83 |
|   doyeol   |  nan  |  nan  |
|   dobok    |  0.0  |  0.0  |
|   gyulju   |  1.83 |  1.91 |
|   bujin    |  0.0  |  0.0  |
+------------+-------+-------+
2022-11-08 16:58:57,315 - mmseg - INFO - Summary:
2022-11-08 16:58:57,316 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 91.03 | 33.31 | 35.86 |
+-------+-------+-------+
2022-11-08 16:58:57,318 - mmseg - INFO - Iter(val) [406]	aAcc: 0.9103, mIoU: 0.3331, mAcc: 0.3586, IoU.background: 0.7544, IoU.normal: 0.8926, IoU.doyeol: nan, IoU.dobok: 0.0000, IoU.gyulju: 0.0183, IoU.bujin: 0.0000, Acc.background: 0.7855, Acc.normal: 0.9883, Acc.doyeol: nan, Acc.dobok: 0.0000, Acc.gyulju: 0.0191, Acc.bujin: 0.0000
2022-11-08 16:59:08,109 - mmse

...

2022-11-08 17:16:29,605 - mmseg - INFO - per class results:
2022-11-08 17:16:29,607 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background |  88.7 | 94.83 |
|   normal   | 92.74 | 97.82 |
|   doyeol   |  nan  |  nan  |
|   dobok    |  0.0  |  0.0  |
|   gyulju   |  9.57 | 10.02 |
|   bujin    |  0.0  |  0.0  |
+------------+-------+-------+
2022-11-08 17:16:29,607 - mmseg - INFO - Summary:
2022-11-08 17:16:29,608 - mmseg - INFO - 
+-------+------+-------+
|  aAcc | mIoU |  mAcc |
+-------+------+-------+
| 94.26 | 38.2 | 40.53 |
+-------+------+-------+
2022-11-08 17:16:29,609 - mmseg - INFO - Iter(val) [406]	aAcc: 0.9426, mIoU: 0.3820, mAcc: 0.4053, IoU.background: 0.8870, IoU.normal: 0.9274, IoU.doyeol: nan, IoU.dobok: 0.0000, IoU.gyulju: 0.0957, IoU.bujin: 0.0000, Acc.background: 0.9483, Acc.normal: 0.9782, Acc.doyeol: nan, Acc.dobok: 0.0000, Acc.gyulju: 0.1002, Acc.bujin: 0.0000
2022-11-08 17:16:38,703 - mmseg - I

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 406/406, 14.1 task/s, elapsed: 29s, ETA:     0s

2022-11-08 17:19:29,631 - mmseg - INFO - per class results:
2022-11-08 17:19:29,637 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 87.92 | 93.91 |
|   normal   | 92.66 | 97.71 |
|   doyeol   |  nan  |  nan  |
|   dobok    |  0.0  |  0.0  |
|   gyulju   | 15.42 | 17.41 |
|   bujin    |  0.84 |  0.86 |
+------------+-------+-------+
2022-11-08 17:19:29,638 - mmseg - INFO - Summary:
2022-11-08 17:19:29,640 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 94.12 | 39.37 | 41.98 |
+-------+-------+-------+
2022-11-08 17:19:29,642 - mmseg - INFO - Iter(val) [406]	aAcc: 0.9412, mIoU: 0.3937, mAcc: 0.4198, IoU.background: 0.8792, IoU.normal: 0.9266, IoU.doyeol: nan, IoU.dobok: 0.0000, IoU.gyulju: 0.1542, IoU.bujin: 0.0084, Acc.background: 0.9391, Acc.normal: 0.9771, Acc.doyeol: nan, Acc.dobok: 0.0000, Acc.gyulju: 0.1741, Acc.bujin: 0.0086
2022-11-08 17:19:38,426 - mmse

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 406/406, 15.2 task/s, elapsed: 27s, ETA:     0s

2022-11-08 17:22:27,060 - mmseg - INFO - per class results:
2022-11-08 17:22:27,062 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 87.71 | 95.28 |
|   normal   | 92.54 |  97.0 |
|   doyeol   |  nan  |  nan  |
|   dobok    |  0.0  |  0.0  |
|   gyulju   |  19.0 | 23.08 |
|   bujin    |  0.0  |  0.0  |
+------------+-------+-------+
2022-11-08 17:22:27,063 - mmseg - INFO - Summary:
2022-11-08 17:22:27,064 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 94.02 | 39.85 | 43.07 |
+-------+-------+-------+
2022-11-08 17:22:27,070 - mmseg - INFO - Iter(val) [406]	aAcc: 0.9402, mIoU: 0.3985, mAcc: 0.4307, IoU.background: 0.8771, IoU.normal: 0.9254, IoU.doyeol: nan, IoU.dobok: 0.0000, IoU.gyulju: 0.1900, IoU.bujin: 0.0000, Acc.background: 0.9528, Acc.normal: 0.9700, Acc.doyeol: nan, Acc.dobok: 0.0000, Acc.gyulju: 0.2308, Acc.bujin: 0.0000


In [13]:
datasets = [build_dataset(cfg.data.train)]
model = build_segmentor(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

2022-11-08 17:22:48,580 - mmseg - INFO - Loaded 3246 images


In [14]:
# Train knet
knet =  train_segmentor(model, datasets, cfg, distributed=False, validate=True)

AssertionError: 

In [ ]:
#model.save('knet.h5')